## Saving pydantic-validated data to SQLite database with SQLModel

Note: not to use list[Any] for building pydantic class model, because there is no type list for sqlalchemy, if you want to save data to database via SQLModel ORM

Reference: 
1. Create a SQLModel Model https://sqlmodel.tiangolo.com/#create-a-sqlmodel-model & https://sqlmodel.tiangolo.com/#sqlalchemy-and-pydantic

In [1]:
%pip install requests
%pip install lxml
%pip install pandas
%pip install watermark
%pip install pydantic
# %pip install pydantic-core==2.6.3
%pip install sqlmodel
%pip install tqdm
%pip install word2number

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

#### Scrape the website content

In [2]:
from pydantic import (BaseModel, 
                      validate_call,
                      computed_field, 
                      field_serializer,
                      HttpUrl,
                      )
from sqlmodel import Field, SQLModel, Session, create_engine, select
from typing import Optional

class BookModel(SQLModel, table=True):
  id: Optional[int] = Field(default=None, primary_key=True)
  price: str
  book_title: str
  book_url: str
  star_rating_class: str

  @computed_field
  @property
  def star_rating(self) -> str:
    return self.star_rating_class.replace("star-rating ","")

  @field_serializer('star_rating')
  def serialize_star_rating(star_rating: str) -> int:
    return w2n.word_to_num(star_rating)

engine = create_engine("sqlite:///database.db")
SQLModel.__table_args__ = {'extend_existing': True} # to avoid error: https://github.com/tiangolo/sqlmodel/issues/350
SQLModel.metadata.create_all(engine)

In [6]:
%%time

import requests
from lxml import html
from word2number import w2n # convert number words (eg. twenty one) to numeric digits (21)
import pandas
from typing import Union # add type hint
from tqdm import tqdm # for displaying a smart progress meter in loops
import sqlalchemy

class BookSpider:
    def __init__(self):
        self.base_url: AnyHttpUrl = "https://books.toscrape.com"
        self.session: requests.sessions.Session  = requests.Session()
        self.headers: dict[str, str] = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
            }
        self.bookmodel = BookModel

    @validate_call(validate_return=True)
    def parse(self, page_num: int=50) -> None:
        # loop the page number to scrape
        
        for page_num in tqdm(range(1,page_num)):
            url = self.base_url + f"/catalogue/page-{page_num}.html"
            output_data = self._fetch_data(url=url)
            # add book data to database rows
            _ = self.create_bookmodel(output_data)

    @validate_call(validate_return=True)
    def _fetch_data(self, url: HttpUrl) -> list[dict]:
        response = self.session.request("GET", url)
        tree = html.fromstring(response.content)      
        
        price = tree.xpath("//article[@class='product_pod']//div[@class='product_price']\
            /p[@class='price_color']/text()")        
        book_title = tree.xpath("//article[@class='product_pod']/h3/a/text()")
        book_url = tree.xpath("//article[@class='product_pod']/h3/a/@href")
        star_rating_class = tree.xpath("//article[@class='product_pod']/p/@class")

        data_list = {
                    "price": price,
                    "star_rating_class": star_rating_class,
                    "book_url": book_url, 
                    "book_title": book_title,
                }
        # change data format from dict[str, list] to list[dict]
        data = pandas.DataFrame(data_list).to_dict(orient="records")
        return data

    @validate_call(validate_return=True)
    def create_bookmodel(self, books_data: list[dict]) -> None:
        with Session(engine) as session:
            for book in books_data:
                bookmodel_data = self.bookmodel.model_validate(book)
                session.add(bookmodel_data)        
                session.commit()
                session.refresh(bookmodel_data)


book_spider = BookSpider()
_ = book_spider.parse()

100%|██████████| 49/49 [00:05<00:00,  8.56it/s]

CPU times: user 1.21 s, sys: 345 ms, total: 1.56 s
Wall time: 5.73 s


In [7]:
import pandas

with Session(engine) as session:
    sql_statement = select(BookModel)
    print(sql_statement)
    df = pandas.read_sql_query(sql_statement, con=engine, index_col=['id'])
df

SELECT bookmodel.id, bookmodel.price, bookmodel.book_title, bookmodel.book_url, bookmodel.star_rating_class 
FROM bookmodel


,price,book_title,book_url,star_rating_class
id,,,,
1,£51.77,A Light in the ...,a-light-in-the-attic_1000/index.html,star-rating Three
2,£53.74,Tipping the Velvet,tipping-the-velvet_999/index.html,star-rating One
3,£50.10,Soumission,soumission_998/index.html,star-rating One
4,£47.82,Sharp Objects,sharp-objects_997/index.html,star-rating Four
5,£54.23,Sapiens: A Brief History ...,sapiens-a-brief-history-of-humankind_996/index...,star-rating Five
...,...,...,...,...
2936,£40.44,Icing (Aces Hockey #2),icing-aces-hockey-2_25/index.html,star-rating Four
2937,£45.24,"Hawkeye, Vol. 1: My ...",hawkeye-vol-1-my-life-as-a-weapon-hawkeye-1_24...,star-rating Three
2938,£34.96,Having the Barbarian's Baby ...,having-the-barbarians-baby-ice-planet-barbaria...,star-rating Four


## Computing environment

In [ ]:
%load_ext watermark

%watermark

# print out pypi packages used
%watermark --iversions

# date
%watermark -u -n -t -z